In [1]:
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.models import Sequential, load_model
from tensorflow.keras.layers import BatchNormalization
from keras.layers.core import Activation, Dense, Dropout
from keras.layers.advanced_activations import PReLU
import numpy as np
import pandas as pd
import pandas_profiling as pdp
import matplotlib.pyplot as plt
import seaborn as sns
import models
from models.evaluation import cross_validation_predict, cross_validation_score
from sklearn.metrics import accuracy_score
from utils import load_datasets, load_target, save_submission
import json
from keras.utils import np_utils
sns.set_style('darkgrid')
%matplotlib inline


In [2]:
config = json.load(open('./config/default.json'))
# X_train, X_test = load_datasets(["Age", "AgeSplit", "EducationNum"])
X_train, X_test = load_datasets(config['features'])
y_train = load_target('target')

In [3]:
scaler = StandardScaler()
scaler.fit(X_train)

patience = 1
layers = 2
dropout = 0.1
units = 3
nb_epoch = 3
batch_size = 128
model = Sequential()
model.add(Dense(units, input_shape=(X_train.shape[1], )))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(dropout))

for l in range(layers - 1):
    model.add(Dense(units))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(dropout))

model.add(Dense(10))
model.add(Activation('softmax'))
early_stopping = EarlyStopping(
    monitor='loss', patience=patience, verbose=0, restore_best_weights=True)
model.compile(loss='categorical_crossentropy',
                optimizer='adagrad',  metrics=['accuracy'])


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-02-21 13:36:07.218584: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-21 13:36:07.219312: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-21 13:36:07.219779: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
history = model.fit(scaler.transform(X_train), np_utils.to_categorical(y_train, num_classes=10),
                    epochs=nb_epoch, batch_size=batch_size, verbose=0, callbacks=[early_stopping])


2022-02-21 13:36:08.409676: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [5]:
y_train.shape

(200000,)

In [6]:
model.predict(scaler.transform(X_train))

2022-02-21 13:37:34.504854: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


array([[0.09510082, 0.10737685, 0.10184819, ..., 0.13926855, 0.09612375,
        0.12404416],
       [0.09833889, 0.12260668, 0.077753  , ..., 0.05209005, 0.0638243 ,
        0.06963198],
       [0.09910155, 0.10849151, 0.08987818, ..., 0.05906248, 0.08090697,
        0.07735704],
       ...,
       [0.09230451, 0.08496595, 0.11231453, ..., 0.09930103, 0.11750078,
        0.10690008],
       [0.09400493, 0.1164024 , 0.10328381, ..., 0.11457378, 0.08702762,
        0.12048645],
       [0.08362278, 0.09611663, 0.05434868, ..., 0.02585233, 0.04709552,
        0.03939295]], dtype=float32)

In [7]:
np_utils.to_categorical(y_train, num_classes=10)


array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)